# Analisis de prestaciones por diagnostico

- En este apartado sequiere ver la cantidad de prestaciones que se realizan por cada diagnostico principal.

- El resultado que se quiere obtener es parecido al siguiente:

|Año|Diagnostico|Resumen Procedimientos|
|---|-----------|----------------------|
|2019|"I25.1"|{"45.23": 20, "73.59": 10}|
|2020|"I25.1"|{"45.23": 7, "73.59": 3}|

In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

from src.features import build_features

warnings.filterwarnings("ignore")
sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

In [4]:
df_procesada = pd.read_csv("../data/processed/df_procesada.csv", sep=";")

In [16]:
procedimientos_long = pd.melt(
    df_procesada,
    id_vars=["COD_HOSPITAL", "ANIO_EGRESO", "DIAGNOSTICO1"],
    value_vars=[f"PROCEDIMIENTO{i}" for i in range(1, 31)],
)

In [19]:
conteo_procedimientos = procedimientos_long.groupby(["ANIO_EGRESO", "COD_HOSPITAL"])[
    "value"
].value_counts()

In [23]:
conteo_procedimientos = conteo_procedimientos.reset_index(name="conteo")

In [27]:
conteo_procedimientos = conteo_procedimientos.sort_values(["ANIO_EGRESO", "value", "conteo"], ascending=False)

In [31]:
conteo_procedimientos["ranking"] = conteo_procedimientos.groupby(["ANIO_EGRESO", "value"]).cumcount() + 1

In [34]:
conteo_procedimientos.query("value == 99.99")

,ANIO_EGRESO,COD_HOSPITAL,value,conteo,ranking
181298,2021.0,113180,99.99,4,1
161843,2021.0,108100,99.99,2,2
149556,2021.0,103100,99.99,1,3
153496,2021.0,105100,99.99,1,4
167314,2021.0,110100,99.99,1,5
168560,2021.0,110120,99.99,1,6
180481,2021.0,113130,99.99,1,7
184007,2021.0,114101,99.99,1,8
189075,2021.0,115107,99.99,1,9
189863,2021.0,115110,99.99,1,10
